## Validation of JTAG Jitter Measurement

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = r"d:\data_store"

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
scope = cw.scope(bitstream = r"firmwares\cwhusky_top.bit")
target = cw.target(scope, cw.targets.SimpleSerial2)

In [ ]:
scope.default_setup()

In [ ]:
def jtag_clkout(enabled):
    if enabled:
        data = 0x08
    else:
        data = 0x00

    CODE_READ = 0x80
    CODE_WRITE = 0xC0
    scope.userio.oa.sendMessage(CODE_WRITE, "USERIO_DEBUG_DRIVEN", [data])
    
    # Can use this to check write worked
    #scope.userio.oa.sendMessage(CODE_READ, "USERIO_DEBUG_DRIVEN")
    

In [ ]:
#Set some lower defaults so ADC doesn't complain later
scope.clock.adc_mul = 1

## 15 MHz CPU, AES, Clock Output

In [ ]:
# program firmware onto target
prog = cw.programmers.SAM4SProgrammer
cw.program_target(scope, prog, "firmwares/fw-15mhz-clkout.hex")

In [ ]:
scope.io.target_pwr = False
time.sleep(0.1)
scope.io.target_pwr = True

In [ ]:
# Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

In [ ]:
# When device is running at 2 MHz baud calculation is off - measuring actual baud shows this is correct:
# Uncomment this when compiling for 2 MHz firmware
#target.baud = 62750

In [ ]:
scope.clock.freq_ctr

In [ ]:
scope.clock.clkgen_src = "extclk"
scope.io.hs2 = None
scope.clock.pll.set_outfreqs(15*1E6/2, 30E6, 1, True)

In [ ]:
#scope.clock.clkgen_src = "extclk"
#scope.io.hs2 = None
#scope.clock.pll.set_outfreqs(15*1E6/2, 15E6, 1, True)

## JTAG Setup

#### Setting up JTAG into Bypass Mode

In [ ]:
def read_tdo_status():
    pins = scope.userio.status
    if pins & (1<<3):
        return True
    else:
        return False
    
def write(tms, tdi):
    old = scope.userio.drive_data
    old &= ~(1<<6 | 1<<7)
    if tms:
        old |= 1<<6
    if tdi:
        old |= 1<<7
    
    scope.userio.drive_data = old
    scope.userio.drive_data = old | (1<<5)
    scope.userio.drive_data = old & ~(1<<5)

Normally `JTAGSEL` being low works fine. Sometimes it's helpful to set it high (call the following function with `True`) for testing. Note when `JTAGSEL` is high code won't run on the microcontroller. But the bypass mode worked fine with this set `False`.

This assumes you've modified the SAM4S2AA board to route TIO3 to JTAGSEL, see this photo:


Note this should NOT be required, so you can recreate the results with a stock ChipWhisperer-Husky kit.

In [ ]:
def change_jtag_mode(boundary_scan=False):
    scope.io.tio3 = boundary_scan
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = True

In [ ]:
#change_jtag_mode(False)

The following requires you to put the 20-pin connector from the USERIO pins on the CW-HUSKY to the JTAG header on the CW313 target board. With that connected, you have the following:

* d[2] = nrst
* d[3] = tdo
* d[4] = rclk
* d[5] = tck
* d[6] = tms
* d[7] = tdi

Running the following code will enable bypass mode and then tri-state the TCK & TDI pins. You can then feed a 40 MHz clock into TCK & a 20 MHz clock into TDI, be sure there is a 90 degree phase offset so the rising edge of the 40 MHz clock correctly clocks the 20 MHz clock into TDI.

If it worked, you should see a 20 MHz clock coming out of TDO. The 20 MHz clock is what we use to measure the delay in the target device.

In [ ]:
def setup_bypass(verbose=True):
    #Take control of TDI, TMS, TCK
    scope.userio.direction = 0b11100000
    
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)

    write(0, 1) #
    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    #Send a bunch of 1's to force bypass mode
    for i in range(0, 10):
        write(0, 1)

    #exit shift-IR state
    write(1,1)

    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    for i in range(0, 10):
        write(0, 0)

    tdo_result = []

    for i in range(0, 10):
        tdo_result.append(read_tdo_status())
        if i == 0:
            write(0, 1)
        else:
            write(0, 0)

    if tdo_result[0:10] == [False, True, False, False, False, False, False, False, False, False]:
        if verbose:
            print("JTAG Setup successful - bypass mode enabled, saw '1' sequence successfuly")
        return True
    else:
        if verbose:
            print("JTAG Setup not successful")
            print(tdo_result)
        return False

In [ ]:
jtag_clkout(False)
setup_bypass()

The following will turn on the TDI/TCK clock. If you externally jumper TDO to HS1 you can confirm the clock is coming back OK.

In [ ]:
jtag_clkout(True)

In [ ]:
scope.clock

## ChipWhisperer Measurement Setup

In [ ]:
jtag_clkout(False)

In [ ]:
setup_bypass()

In [ ]:
jtag_clkout(True)

In [ ]:
scope.gain.mode = "low"
scope.gain.gain = 10

In [ ]:
scope.io.nrst = None

### TVLA for Jitter Measurement

We'll now capture 3 T-Test datasets:

1. Where the JTAG bypass loop is enabled, and a clock is fed in
2. Where the JTAG bypass loop is enabled, but the clock is stopped (this will force the I/O pin to a constant state)
3. Where the JTAG bypass loops is disabled, but the clock is running (don't expect good results, but a good baseline)

In [ ]:
scope.adc.samples = 10000

In [ ]:
scope.clock

In [ ]:
scope.gain.gain = 45

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
scope.clock

**Dataset #1:** The JTAG clock was turned on earlier, so we just do a capture.

In [ ]:
group1, group2 = capture_ttest(10000, splot=splot)

In [ ]:
#nptsave("cwhusky_jtag_mixer_15mhzcpu_30mhzadc_10ktraces_sync", group1, group2)

**Dataset #2:** Turn off the clock, which will leave TDO driving a constant value and still "connected" to the bypass register.

In [ ]:
jtag_clkout(False)

In [ ]:
group1, group2 = capture_ttest(10000, splot=splot)

In [ ]:
#nptsave("cwhusky_jtag_mixer_clockoff_15mhzcpu_30mhzadc_10ktraces_sync", group1, group2)

**Dataset #3:** Power cycle the target, which will force it back to a "normal power up state" as if we never did any JTAG stuff. Could also do a JTAG reset (send TMS/TDI high five times), but we do this in case other stuff happened.

In [ ]:
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
# Turn clock back on
jtag_clkout(True)

In [ ]:
group1, group2 = capture_ttest(10000, splot=splot)

In [ ]:
#nptsave("cwhusky_jtag_mixer_tclknobypass_15mhzcpu_30mhzadc_10ktraces_sync", group1, group2)

#### In-Notebook Analysis (mostly for debug)

The T-Test results notebook is used for plotting, this is used for checking averages and results when you can't wait.

In [ ]:
import numpy as np
mean1 = np.mean(group1, axis=0)#[2000:]
mean2 = np.mean(group2, axis=0)#[2000:]
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

In [ ]:
plt.figure()
plot_t(t_val, 10000, "Debug")
plt.show()

## CPA Measurements

### With Mixer, with TDI/TDO Clock

In [ ]:
scope.gain.gain = 45

In [ ]:
jtag_clkout(False)

In [ ]:
setup_bypass()

In [ ]:
jtag_clkout(True)

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 100000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace
        
    if i % 10000 == 0:
        raise IOError("Change 'False' to True and run this again")
        try:
            print("Attempting save at {:d}".format(i))            
            save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_sync_k.ets", overwrite=False)
            print("phew")
        except:
            print("Hmm... save failed, skipped!")
            pass

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_sync_k.ets", overwrite=True)

### Baseline with clock off (I/O pin measurement)

In [ ]:
# Turn clock off
jtag_clkout(False)

In [ ]:
scope.gain.gain = 65

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 100000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace
        
    if i % 10000 == 0:
        raise IOError("Change 'False' to True and run this again, comment out this line")
        try:
            print("Attempting save at {:d}".format(i))
            save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets", overwrite=False)
            print("phew")
        except:
            print("Hmm... save failed, skipped!")
            pass

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets", overwrite=True)

## Baseline for CPA

In [ ]:
# Turn clock off
jtag_clkout(False)

In [ ]:
scope.adc

In [ ]:
scope.gain.gain = 50

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 5000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_shunt_15mhzcpu_30mhzadc_sync.ets", overwrite=True)

In [ ]:
group1, group2 = capture_ttest(10000, splot=splot)

In [ ]:
#nptsave("cwhusky_shunt_15mhzcpu_30mhzadc_10ktraces_sync", group1, group2)

In [ ]:
target

In [ ]:
# Turn clock off
jtag_clkout(True)

In [ ]:
scope.gain.gain = 50

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 5000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_shunt_withtditck_15mhzcpu_30mhzadc_sync.ets", overwrite=False)

## Countermeasure Testing

### Countermeasure Description

The countermeasure is run on a TinyFPGA BX. It uses a 148 MHz clock to "reclock" the TDO line. This doesn't seem to impact usage of JTAG tools at normal speeds (250kHz - 10 MHz).


```
module top (
   input  CLK,   // 16 MHz on-board clock
   output LED,   // on-board LED
   output USBPU,  // USB pull-up enable, set low to disable

   input PIN_2,
   output PIN_3,
   output PIN_13


   );

    /**
    * PLL configuration
    *
    * This Verilog module was generated automatically
    * using the icepll tool from the IceStorm project.
    * Use at your own risk.
    *
    * Given input frequency:        16.000 MHz
    * Requested output frequency:  150.000 MHz
    * Achieved output frequency:   148.000 MHz
    */
    wire clock_in;
    wire clock_out;
    wire locked;
    SB_PLL40_CORE #(
                    .FEEDBACK_PATH("SIMPLE"),
                    .DIVR(4'b0000),         // DIVR =  0
                    .DIVF(7'b0100100),      // DIVF = 36
                    .DIVQ(3'b010),          // DIVQ =  2
                    .FILTER_RANGE(3'b001)   // FILTER_RANGE = 1
            ) uut (
                    .LOCK(locked),
                    .RESETB(1'b1),
                    .BYPASS(1'b0),
                    .REFERENCECLK(clock_in),
                    .PLLOUTCORE(clock_out)
                    );


  reg relatched;
  wire clock;

  //PLL connection
  assign clock_in = CLK;
  assign clock = clock_out;

  //Countermeasure OFF - uncomment this line, comment out the below
  //assign relatched = PIN_2;


  //Countermeasure ON - uncomment this line, comment out hte above
  always @(posedge clock) relatched <= PIN_2;

   assign PIN_3 = relatched;
   assign PIN_13 = relatched;


   assign LED = 1'b1;  // blink on-board LED every second
   assign USBPU = 1'b0;   // disable USB
endmodule  // top
```

To program the board, `tinyprog` is used:

```
!pip install tinyprog
```

Note the board is powered from 3.3V from the target device. The USB is only connected during programming.

### CW Setup

In [ ]:
scope.clock

### Countermeasure - FPGA inserted, but direct connection (no reclocking)

The second baseline is where the TinyFPGA is inserted into the TDO pin, but the countermeasure is "disabled" by just doing this in the FPGA design:

```
assign relatched = PIN_21;
```

This means no reclocking is happening. This is to check how much jitter is added just by the additional digital path.

In [ ]:
!tinyprog -p "firmwares/ice40_countermeasure_off_jtagcable.bin" -b

In [ ]:
jtag_clkout(False)
setup_bypass()

In [ ]:
jtag_clkout(True)

In [ ]:
scope.gain.gain = 45
scope.gain.mode = "low"
scope.gain

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
N = 10000
group1, group2 = capture_ttest(N, picoscope=False, splot=splot)

In [ ]:
nptsave("cwhusky_jtag_mixerboard_15mhzcpu_30mhzadc_10ktraces_sync_fpgacountermeasuredisabled", group1, group2)

### Countermeasure Enabled

The same setup as previously, but the FPGA is reprogrammed with the countermeasure.

In [ ]:
!tinyprog -p "firmwares/ice40_countermeasure_on_jtagcable.bin" -b

In [ ]:
jtag_clkout(False)
setup_bypass()

In [ ]:
jtag_clkout(True)

In [ ]:
N = 10000
group1, group2 = capture_ttest(N, picoscope=False, splot=splot)

In [ ]:
nptsave("cwhusky_jtag_mixerboard_15mhzcpu_30mhzadc_10ktraces_sync_fpgacountermeasureenabled", group1, group2)

In [ ]:
scope.gain.gain = 30
scope.gain.mode = "low"
scope.gain

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 100000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace
        
    if i % 10000 == 0:
        try:
            print("Attempting save at {:d}".format(i))
            raise IOError("Change 'False' to True and run this again")
            save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtagcounetmeasured_mixer_15mhzcpu_30mhzadc_sync_k.ets", overwrite=False)
            print("phew")
        except:
            print("Hmm... save failed, skipped!")
            pass

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/cwhusky_jtagcounetmeasured_mixer_15mhzcpu_30mhzadc_sync_k.ets", overwrite=True)

In [ ]:
scope.clock.clkgen_src = "extclk"
scope.io.hs2 = None
scope.clock.pll.set_outfreqs(15*1E6/2, 15E6, 2, True)